## Preprocessing Data Notebook

In [1]:
import os, sys
import pandas as pd
from multiprocessing import Process, Manager
import numba

sys.path.insert(0, 'src')
from fs_utils import remove_and_create_dir
from config import BaseConfig

In [7]:
# def _get_traffic(dst_data_dir: str, 
#                  features: dict, 
#                  config: BaseConfig,
#                  file: str):
#     '''
#     Obtain the traffic features from csv provided at file and save to dst_data_dir
    
#     Arg(s):
    
    
#     '''
    
#     df = pd.read_csv(file).fillna(config.sp_wgt)
#     print(df.head)
    
    
# def _extract_features(data_root: str, config: BaseConfig):
#     for mode in ['train', 'test']:
#         src_data_dir = os.path.join(data_root, '_{}'.format(mode))
        
#         files = []
#         # Enumerate through data directory and save file paths to all csvs that are NOT checkpoints
#         for dirpath, _, filenames in os.walk(src_data_dir):
#             for file in filenames:
#                 if (os.path.splitext(file)[1] == '.csv') and 'checkpoint' not in file:
#                     files.append(os.path.join(dirpath, file))
                    
#         dst_data_dir = os.path.join(data_root, '{}'.format(mode))
#         remove_and_create_dir(dst_data_dir)
        
#         print("{} files to process...".format(len(files)))
        
#         features = {}
#         for file in files:
#             _get_traffic(
#                 dst_data_dir=dst_data_dir,
#                 features = features,
#                 config=config,
#                 file=file
        


In [4]:
def multi_process(func, FILES, args=None):
    it = 0
    while True:
        files = FILES[it * config.no_process:(it + 1) *
                      config.no_process]
        if len(files) > 0:
            threads = []
            for file in files:
                ARGS = list(args)
                ARGS.append(file)
                t = Process(target=func, args=tuple(ARGS))
                threads.append(t)
                t.start()
            for thr in threads:
                thr.join()
            it += 1
        else:
            break

def feature_extraction(config: BaseConfig,
                       target: list[str]):
    """feature extraction module"""

    @numba.jit
    def gettraffic(dst_folder, my_fet, file):
        df = pd.read_csv(file).fillna(config.sp_wgt)

        #traffic load features
        ins = feaTure(df, config.no_of_port, config.no_of_buffer,
                      config.window, config.ser_rate)
        C_dst_SET, LOAD = ins.getCount()
        for i in range(config.no_of_port):
            df['TI%i' % i] = LOAD[i]
        for i in range(config.no_of_port):
            for j in range(config.no_of_buffer):
                df['load_dst{}_{}'.format(i, j)] = C_dst_SET[(i, j)]

        #arrival patterns
        df['inter_arr_sys'] = df['timestamp (sec)'].diff()
        if config.no_of_buffer > 1:
            for i in range(config.no_of_buffer):
                t = df[df['priority'] ==
                       i]['timestamp (sec)'].diff().rename(
                           'inter_arr{}'.format(i)).to_frame()
                df = df.join(t)

        #save
        filename = file.split('/')[-1]
        drop_cols = ['timestamp (sec)'] + target
        if config.no_of_buffer == 1: drop_cols += ['priority']
        fet_cols = list(df.columns.drop(drop_cols))
        my_fet['fet_cols'] = fet_cols
        df[fet_cols + target].fillna(method='ffill').dropna().to_csv(
            '{}/{}'.format(dst_folder, filename), index=False)

    with Manager() as MG:
        my_fet = MG.dict()
        for mode in ['train', 'test']:
            file_dir = './data/{}/_traces/_{}'.format(
                config.modelname, mode)
            FILES = []
            for dirpath, dirnames, filenames in os.walk(file_dir):
                for file in filenames:
                    if (os.path.splitext(file)[1]
                            == '.csv') and 'checkpoint' not in file:
                        FILES.append(os.path.join(dirpath, file))

            dst_folder = './data/{}/_traces/{}'.format(
                config.modelname, mode)
            if os.path.exists(dst_folder):
                shutil.rmtree(dst_folder)
            os.makedirs(dst_folder)
            multi_process(gettraffic,
                               FILES,
                               args=(dst_folder, my_fet))
        fet_cols = my_fet['fet_cols']

In [5]:
config = BaseConfig()
target = ['time_in_sys']
data_root = os.path.join('data', 'dqn_data', '4-port switch', 'FIFO', '_traces')

# _extract_features(
#     data_root=data_root,
#     config=config)

feature_extraction(
    config=config,
    target=target)

NameError: name 'self' is not defined